In [6]:
import pandas as pd
import numpy as np
from pathlib import Path
from mda_core.time import Time
from mda_core import angle, utils
from mda_core import constants as C
from mda_core import frames
from org.orekit.models.earth import GeoMagneticField,GeoMagneticFieldFactory
from org.orekit.bodies import OneAxisEllipsoid

In [7]:
Flag = 2
if Flag == 1:
    Filepath = r'../data/D2-FC-GeneralTM.csv'
elif Flag == 2:
    Filepath = r'../data/shakuntala-ADCS-20231222.csv'
elif Flag == 3:
    Filepath = r'../data/set7_true_data_sws.csv'
else:
    raise FileExistsError("File Not found!")

In [8]:
tm = pd.DataFrame()
SunMag = pd.DataFrame()
msmt = {
'time':[],
'BField': []
}
Sun_eph = {
'time': [],
'Px': [],
'Py': [],
'Pz': []
}

In [9]:
tm = pd.read_csv(Filepath)
tm.head()

,Version,VersionReserved,timestamp,SunSensor_0_Vector_0,SunSensor_0_Vector_1,SunSensor_0_Vector_2,SunSensor_0_Temperature,SunSensor_0_DataValid,SunSensor_0_TemperatureValid,SunSensor_0_dT_Ms,...,Debug_2,Debug_3,Debug_4,Debug_5,Debug_6,Debug_7,Debug_8,EndMarker,Allign,Unnamed: 309
0,6,0,1703202720,0.0,0.0,0.0,-6.34375,1,1,0,...,0,0,0,0,0,0,0,4099975969,0,NaN
1,6,0,1703202780,0.0,0.0,0.0,-6.84375,1,1,0,...,0,0,0,0,0,0,0,4099975969,0,NaN
2,6,0,1703202840,0.0,0.0,0.0,-7.18750,1,1,0,...,0,0,0,0,0,0,0,4099975969,0,NaN
3,6,0,1703202900,0.0,0.0,0.0,-7.50000,1,1,0,...,0,0,0,0,0,0,0,4099975969,0,NaN
4,6,0,1703202960,0.0,0.0,0.0,-7.93750,1,1,0,...,0,0,0,0,0,0,0,4099975969,0,NaN


In [10]:
igrfyear = GeoMagneticField.getDecimalYear(1,4,2024)
model = GeoMagneticFieldFactory.getWMM(igrfyear)
fr_gcrf = frames.get_earth_GCRF()
earth = OneAxisEllipsoid(C.EARTH_RADIUS,C.EARTH_FLATTENING,frames.get_earth_GCRF())
if Flag == 1:
    for index,sample in tm.iterrows():
        t = Time(pd.to_datetime(sample['timestamp'], unit='s'))
        time = t.get_absolutedate()
        lat_rad = float(angle.deg2rad(sample['gpsLatitude']))
        long_rad = float(angle.deg2rad(sample['gpsLongitude']))
        alt_m = float(sample['gpsAltitudeMeters'])
        igrf = model.calculateField(lat_rad,long_rad,alt_m)
        igrf = igrf.getFieldVector()
        SunP = C.SUN_PV.getPVCoordinates(time,fr_gcrf).getPosition()
        Sun_eph['time'].append(t.get_datetime())
        Sun_eph['Px'].append(SunP.getX())
        Sun_eph['Py'].append(SunP.getY())
        Sun_eph['Pz'].append(SunP.getZ())
        msmt['time'].append(t.get_datetime())
        msmt['BField'].append(igrf)
elif Flag == 2:
    for index,sample in tm.iterrows():
        t = Time(pd.to_datetime(sample['SatelitePosition_UnixTime'],unit='s'))
        time = t.get_absolutedate()
        posx = sample['SatelitePosition_ro_m_0']
        posy = sample['SatelitePosition_ro_m_1']
        posz = sample['SatelitePosition_ro_m_2']
        position = utils.list_to_vector3d([posx,posy,posz])
        gp = earth.transform(position,earth.getBodyFrame(),time)
        lat_rad = float(gp.getLatitude())
        long_rad = float(gp.getLongitude())
        alt_m = float(gp.getAltitude())
        igrf = model.calculateField(lat_rad,long_rad,alt_m)
        igrf = igrf.getFieldVector()
        SunP = C.SUN_PV.getPVCoordinates(time,fr_gcrf).getPosition()
        Sun_eph['time'].append(t.get_datetime())
        Sun_eph['Px'].append(SunP.getX())
        Sun_eph['Py'].append(SunP.getY())
        Sun_eph['Pz'].append(SunP.getZ())
        msmt['time'].append(t.get_datetime())
        msmt['BField'].append(igrf)
elif Flag == 3:
     for index,sample in tm.iterrows():
        t = Time(pd.to_datetime(sample['time']))
        time = t.get_absolutedate()
        posx = sample['Px']
        posy = sample['Py']
        posz = sample['Pz']
        position = utils.list_to_vector3d([posx,posy,posz])
        gp = earth.transform(position,earth.getBodyFrame(),time)
        lat_rad = float(gp.getLatitude())
        long_rad = float(gp.getLongitude())
        alt_m = float(gp.getAltitude())
        igrf = model.calculateField(lat_rad,long_rad,alt_m)
        igrf = igrf.getFieldVector()
        SunP = C.SUN_PV.getPVCoordinates(time,fr_gcrf).getPosition()
        Sun_eph['time'].append(t.get_datetime())
        Sun_eph['Px'].append(SunP.getX())
        Sun_eph['Py'].append(SunP.getY())
        Sun_eph['Pz'].append(SunP.getZ())
        msmt['time'].append(t.get_datetime())
        msmt['BField'].append(igrf)    

Suneph_df = pd.DataFrame(Sun_eph)
Msmt_df = pd.DataFrame(msmt)
Msmt_df.columns

Index(['time', 'BField'], dtype='object')

In [11]:
SunMag['Time'] = Suneph_df['time']
SunMag['Sun PX'] = Suneph_df['Px']
SunMag['Sun PY'] = Suneph_df['Py']
SunMag['Sun PZ'] = Suneph_df['Pz']
SunMag['Bfield (nT)'] = msmt['BField']

In [12]:
SunMag.to_csv('ECIvector.csv',index=False)

In [13]:
igrfyear = GeoMagneticField.getDecimalYear(1,4,2024)
igrfyear
mod = GeoMagneticFieldFactory.getWMM(2024.0)
igrf = mod.calculateField(0.5570752098432994, 0.5061478466338231, 420562.46875)
display(igrf)

<GeoMagneticElements: MagneticField[B={24,608.028; 1,814.723; 26,296.457},H=24,674.851,F=36,060.393,I=0.817,D=0.074]>

In [14]:
len(msmt)

2

In [15]:
igrf.getTotalIntensity()

36060.393236608405

In [16]:
gp.getLatitude()

-0.6900801149673693